# Batch run models

In [ ]:
%load_ext lab_black
import os, multiprocessing, time, papermill, sys, sqlite3
from IPython.display import clear_output
from tqdm import tqdm
from meta import ModelConfig, make_batch_cfg
import pandas as pd

# nest_asyncio.apply()
# Gen OG random seed: [int(random.random() * 1e5) for x in range(10)]

In [ ]:
def csv_to_bigquery(csv_file, dataset_name, table_name):
    from google.cloud import bigquery
    import json, os
    import pandas as pd

    # Create connection to BQ and push data
    client = bigquery.Client()
    dataset = client.create_dataset(dataset_name, exists_ok=True)
    table_ref = dataset.table(table_name)

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1, autodetect=True
    )

    with open(csv_file, "rb") as f:
        job = client.load_table_from_file(f, table_ref, job_config=job_config)

    job.result()
    print(f"Loaded {job.output_rows} rows into {dataset_name}:{table_ref.path}")

In [ ]:
df = query_job.to_dataframe()

In [ ]:
csv_file = "/home/jupyter/tf/models/O2P_baseline_v0/result_grain_item.csv"
csv_to_bigquery(csv_file, dataset_name="boo", table_name="grain")

## Make configs

In [ ]:
batch_name = "error_injection_timing_test"
batch_output_dir = f"models/batch_run/{batch_name}/"

param_grid = {
    "rng_seed": [75429, 32572, 81590, 15999, 75629, 79044, 70079, 41958, 50626, 92328],
    "inject_error_ticks": [2, 11],
}

static_hpar = {
    "tf_root": "/home/jupyter/tf",
    "ort_units": 119,
    "pho_units": 250,
    "sem_units": 2446,
    "hidden_os_units": 500,
    "hidden_op_units": 100,
    "hidden_ps_units": 500,
    "hidden_sp_units": 500,
    "pho_cleanup_units": 20,
    "sem_cleanup_units": 50,
    "pho_noise_level": 0.0,
    "sem_noise_level": 0.0,
    "activation": "sigmoid",
    "tau": 1 / 3,
    "max_unit_time": 4.0,
    "output_ticks": 11,
    "learning_rate": 0.01,
    "zero_error_radius": 0.1,
    "n_mil_sample": 2.0,
    "batch_size": 100,
    "save_freq": 10,
    "batch_name": batch_name,
}

batch_cfgs = make_batch_cfg(
    batch_name, batch_output_dir, static_hpar, param_grid, "hs04_phase2.ipynb"
)

## Push config to database

In [ ]:
cfg_df = pd.DataFrame()

for i, cfg in enumerate(batch_cfgs):
    cfg_df = pd.concat([cfg_df, pd.DataFrame(cfg["params"], index=[i])])


sqlite_file = os.path.join(batch_output_dir, "batch_results.sqlite")
con = sqlite3.connect(sqlite_file)
cur = con.cursor()
cfg_df.to_sql("batch_config", con, if_exists="replace")

## Run batch
- Can run on Jupyter Lab
- Cannot run on VSCode

In [ ]:
# Run
def run_batch(cfg):
    """
    Using papermill to run parameterized notebook
    To prevent overwriting, set default overwrite to False if needed
    """

    print("Running model {}".format(cfg["sn"]))
    os.makedirs(cfg["model_folder"], exist_ok=True)
    papermill.execute_notebook(
        cfg["in_notebook"],
        cfg["out_notebook"],
        parameters=cfg["params"],
    )

    clear_output()


# Run in parallel pool

# with multiprocessing.Pool(2) as pool:
#     pool.map(run_batch, batch_cfgs)


# Run in series
for cfg in tqdm(batch_cfgs):
    run_batch(cfg)



print("Done")

Compile and save results

In [ ]:
# Cannot execute together with parallel run for some reason... maybe due to nest_asynio... forgot how to fix...
# cfgs, df = parse_batch_results(batch_cfgs)
# df.to_csv(batch_output_dir + "bcdf.csv")
# cfgs.to_csv(batch_output_dir + "cfgs.csv")

### Re-evaluation snippet

In [ ]:
# cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(cur.fetchall())
# con.execute('DROP TABLE strain')

In [ ]:
import tensorflow as tf
import meta, data_wrangling, modeling, evaluate

for i in tqdm(range(20)):

    code_name = cfg_df.code_name[i]
    cfg = meta.ModelConfig.from_json(
        os.path.join(batch_output_dir, code_name, "model_config.json")
    )
    data = data_wrangling.MyData()
    model = modeling.HS04Model(cfg)

    # Remove existing strain_mean_df.csv
    try:
        os.remove(
            os.path.join(batch_output_dir, code_name, "eval", "strain_mean_df.csv")
        )

    except:
        pass

    test = evaluate.EvalReading(cfg, model, data)

    test.eval("strain")
    clear_output()

Shutdown compute engine

In [ ]:
time.sleep(30)
print('shuting down')
!sudo poweroff